In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, LSTM
from keras.layers import Embedding
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.layers.merge import Concatenate
import tensorflow as tf
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

Using TensorFlow backend.


## 1. Import data: Combine & Merge Tables

In [2]:
fd = pd.read_csv("full_data.csv")
fd = fd.iloc[:,[0,1,3,4,5,9,10]]
print(fd.shape)

(48979, 7)


In [3]:
tag = pd.read_excel("tagged.xlsx")
extra_tag = pd.read_csv("additional_tags.csv")
all_tag = pd.concat([tag,extra_tag])

In [4]:
merge = fd.merge(all_tag,left_on='product_id',right_on='product_id',how="inner")
merge.shape

(159013, 10)

## 2. Prepare Data

#### Filter out records for Attribute "Occasion"

In [5]:
data = merge[merge["attribute_name"]=="occasion"]
for i in data.columns:
    data[i] = data[i].str.lower()
print(data.shape)

(21263, 10)


In [6]:
# Check unique attribute value for occasion
data['attribute_value'].unique()

array(['work', 'day to night', 'daytonight', 'weekend', 'vacation',
       'nightout', 'coldweather', 'night out', 'workout'], dtype=object)

In [7]:
def regex_cleaning(value):
    new_value = re.sub("nightout",'night out', value)
    new_value1 = re.sub("daytonight", 'day to night', new_value)
    new_value2 = re.sub("coldweather", 'cold weather', new_value1)
    return new_value2
data['attribute_value'] = data['attribute_value'].apply(regex_cleaning)

In [8]:
data['attribute_value'].unique()
# Check again, we got in total 7 different occasion categories

array(['work', 'day to night', 'weekend', 'vacation', 'night out',
       'cold weather', 'workout'], dtype=object)

In [9]:
# Create entity group: Combine product_id and product_color_id
data['id'] = data['product_id']+data['product_color_id']

In [10]:
# Check whether we have duplicate records
data.groupby('id')['attribute_value'].count()

id
01dpc9gstt72khnn0mndnkh7rd01dpc9gstzypv54n9b1tbb6h1x     4
01dpc9gstt72khnn0mndnkh7rd01dpc9gtp1qdcw3exqttr6ad94     4
01dpc9gstt72khnn0mndnkh7rd01dpc9gvg1zn8rrax77s27jrzs     4
01dpcb2keavxxkfvm7fxbne4vy01dpcb2kef1cfywfwa97d3vn11    10
01dpcb2keavxxkfvm7fxbne4vy01dpcb2mmtpctv3rnjv2bbaf28    10
                                                        ..
01e6078g3gratf2c96vkyywsgd01e6078g42kzpg57ncp4yg1tbb     1
01e6078g3gratf2c96vkyywsgd01e6078m3mmsaetkp9pd5dprr2     1
01e6079dg58yw9k78d57c6j2y101e6079dgrr4chkcpkny96g15p     3
01e6079qfkh4hpzfq31t6wdrrx01e6079qg5gtp5jyrcwgrwy4zc     1
01e607bhrqajdz76mjfn7rprk101e607bhsbcwh034nx5tmjh3z7     2
Name: attribute_value, Length: 5411, dtype: int64

In [11]:
# remove duplicate records
data.drop_duplicates(subset =["id",'attribute_name','attribute_value'],
                     keep = "first",inplace = True) 
data.shape

(12625, 11)

#### Create dummies for response variable

In [12]:
dummies = pd.get_dummies(data['attribute_value'])
dummies.head()

,cold weather,day to night,night out,vacation,weekend,work,workout
6,0,0,0,0,0,1,0
20,0,1,0,0,0,0,0
31,0,0,0,0,0,1,0
44,0,1,0,0,0,0,0
47,0,0,0,0,0,1,0


In [13]:
data = pd.concat([data, dummies], axis=1, sort=False)

In [14]:
#distribution of each occasion attribute value
data['attribute_value'].value_counts()

weekend         4159
day to night    3804
work            1872
night out       1210
vacation         973
cold weather     348
workout          259
Name: attribute_value, dtype: int64

In [15]:
# A same product may belongs to multiple occasions
# Therefore, we sum dummies together for each product
response = data.groupby('id').agg({'cold weather':'sum','day to night':'sum',
                                 'night out':'sum','vacation':'sum',
                                 'weekend':'sum','work':'sum','workout':'sum'}).reset_index()
print(response.shape)

(5411, 8)


In [16]:
combine = data.iloc[:,:11].merge(response,left_on='id',right_on='id')

In [17]:
combine.drop_duplicates(subset =["id"],keep = "first",inplace = True) 
combine.shape

(5411, 18)

In [18]:
combine = combine.set_index('id')

#### Preprocess Feature columns

In [19]:
# drop records with both null value in description and details
combine.dropna(subset=["description", "details"], how='all',inplace =True)

In [20]:
combine.isnull().sum()

product_id               0
brand                    0
product_full_name        0
description            380
brand_category         337
brand_canonical_url      0
details                397
product_color_id         0
attribute_name           0
attribute_value          0
cold weather             0
day to night             0
night out                0
vacation                 0
weekend                  0
work                     0
workout                  0
dtype: int64

In [21]:
# Combine details and description into one "Text" column
combine= combine.replace(np.nan, 'UNKNOWN_TOKEN', regex=True)
combine['details'] = combine['details'].str.replace("\n", "")
combine['text'] = combine['description']+' '+combine['details']

In [22]:
# Preprocess for Text info
import spacy
nlp = spacy.load('en_core_web_md')

def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    #remove stopwords and do lemmatization
    doc = nlp(sentence)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    return " ".join(tokens)

In [23]:
combine['text'] = combine['text'].apply(preprocess_text)
combine['product_full_name'] = combine['product_full_name'].apply(preprocess_text)
combine['brand_category'] = combine['brand_category'].apply(preprocess_text)

In [24]:
def preprocess_url(url):
    # Remove http//:www.
    url = re.sub('https://www.', '', url)
    # Remove .com
    url = re.sub('.com', '', url)
    # Remove separator "/"
    url = re.sub('/', ' ', url)
    # Remove separator "-"
    url = re.sub('-', ' ', url)
    # Remove numbers
    url = re.sub(r'[0-9]+', ' ', url)
    # Remove Single character
    url = re.sub(r"\s+[a-zA-Z]\s+", ' ', url)
    # Removing multiple spaces
    url = re.sub(r'\s+', ' ', url)
    # Remove stopwords and do lemmatization
    doc = nlp(url)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    return " ".join(tokens)

In [25]:
combine['brand_canonical_url'] = combine['brand_canonical_url'].apply(preprocess_url)

In [26]:
# Combine all brand info together
combine['brand_info'] = combine['brand']+' '+combine['product_full_name']+' '+\
                        combine['brand_category']+' '+combine['brand_canonical_url']

In [27]:
# Filter out useful columns
df = combine.loc[:,["text","brand_info","cold weather","day to night","night out",
                 "vacation","weekend","work","workout"]]

In [28]:
# check one product
df[df.index=="01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs2pz6nj"]

,text,brand_info,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,,,
01dtjcerf6f4nrz2wsjffa1eys01dtjcergehdmq5fajqs2pz6nj,beige stretch silk slip silk spandex dry clean...,theory teah stretch silk camisole clothing top...,0,1,1,0,1,1,0


____

## 3. Embedding Model: Description & Detail

In [29]:
docs = df['text']
labels = df.iloc[:,2:]
X = docs
y = labels.values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3,random_state=42)

In [30]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1
vocab_size

4487

In [31]:
from keras.preprocessing.text import text_to_word_sequence
from typing import List
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))
max_length = get_max_token_length_per_doc(docs)
max_length

165

In [32]:
X_train_pad = pad_sequences(X_train_token, padding='post', maxlen=max_length)
X_test_pad = pad_sequences(X_test_token, padding='post', maxlen=max_length)

In [33]:
X_train_pad.shape

(3779, 165)

In [34]:
# using GloVe word embeddings to convert text inputs 
# to their numeric counterparts.

def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [35]:
EMBEDDING_SIZE = 100
embedding_matrix = zeros((vocab_size, EMBEDDING_SIZE))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [36]:
# Predictive model
deep_inputs = Input(shape=(max_length,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
flatten_layer = Flatten()(embedding_layer)
dense_layer_1 = Dense(100,activation='relu')(flatten_layer)
dense_layer_2 = Dense(7, activation='sigmoid')(dense_layer_1)
embedding_model = Model(inputs=deep_inputs, outputs=dense_layer_2)

embedding_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [37]:
embedding_model.fit(X_train_pad, y_train, epochs=10, verbose=1)

Epoch 1/10
3779/3779 [==============================] - 1s 301us/step - loss: 0.4192 - acc: 0.8178
Epoch 2/10
3779/3779 [==============================] - 1s 251us/step - loss: 0.2851 - acc: 0.8800
Epoch 3/10
3779/3779 [==============================] - 1s 258us/step - loss: 0.2095 - acc: 0.9189
Epoch 4/10
3779/3779 [==============================] - 1s 250us/step - loss: 0.1502 - acc: 0.9482
Epoch 5/10
3779/3779 [==============================] - 1s 253us/step - loss: 0.1079 - acc: 0.9670
Epoch 6/10
3779/3779 [==============================] - 1s 253us/step - loss: 0.0786 - acc: 0.9795
Epoch 7/10
3779/3779 [==============================] - 1s 256us/step - loss: 0.0630 - acc: 0.9862
Epoch 8/10
3779/3779 [==============================] - 1s 252us/step - loss: 0.0484 - acc: 0.9898
Epoch 9/10
3779/3779 [==============================] - 1s 253us/step - loss: 0.0387 - acc: 0.9922
Epoch 10/10
3779/3779 [==============================] - 1s 261us/step - loss: 0.0337 - acc: 0.9932


In [38]:
# Predict on test dataset
score = embedding_model.evaluate(X_test_pad, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

1620/1620 [==============================] - 0s 69us/step
Test Score: 0.47880460531623276
Test Accuracy: 0.8721339106559753


In [39]:
cols_target = ["cold weather","day to night","night out","vacation","weekend","work","workout"]
pred = embedding_model.predict(X_test_pad)
embedding_result = pd.DataFrame(data= pred, columns = cols_target,index=X_test.index)
embedding_result

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e600eckpav431bzqjem5v4wy01e600eg7dzgakzjghgkrfvyb3,0.023494,0.999534,0.000009,1.388862e-02,0.999915,0.003947,1.300395e-07
01e1jm16ncm11qc1gsmkh3erch01e1jm16ns1wtkcph80zeswkx3,0.008819,0.877952,0.343095,1.700809e-03,0.579749,0.052660,2.688225e-04
01e2ky9h2267fxc9d382f077jd01e2ky9h4j0ffd2zcf826e8zm5,0.957473,0.482324,0.009063,6.284370e-05,0.367169,0.999456,1.773343e-04
01e4edbbqfaxnjdz93scvnnw8n01e4edbbqsx9zk71wd5vqe5yqe,0.001779,0.001259,0.004847,7.333902e-01,0.998459,0.000370,1.728570e-06
01e2m0qx4j85htkq2136q0tbvb01e2m0qx4xx4zpm06rj5888gzk,0.000307,0.153232,0.007264,1.128590e-01,0.680969,0.011169,6.006781e-01
...,...,...,...,...,...,...,...
01e2m4heymjm0f3ygprsxz8c0d01e2m4hez04zkv0h81824s23zt,0.002007,0.999479,0.932751,2.583727e-02,0.799184,0.259645,2.188816e-04
01e2kyfmf75neaxyhxc8vypad801e2kyfsrwnxya6y39c6d7gvk8,0.079482,0.982094,0.088442,1.453407e-04,0.033204,0.985668,5.141476e-04
01e4rts300d1qvsgtedt3kp7wz01e4rts309b93wvg0vj2krscwc,0.000199,0.999991,0.000318,1.029118e-04,0.960954,0.006152,1.154691e-04


In [40]:
embedding_model.save("occasion_embedding_model.h5")

____

## 4. Vectorization Model: Brand, Name, URL, Brand Category

In [41]:
info = df['brand_info']
labels = df.iloc[:,2:]
X = info
y = labels

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3,random_state=42)

In [42]:
## TF-IDF Vectorizer
train_X = list(X_train.values)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=3, 
                             max_df=0.9,
                             token_pattern=r'\w{3,}',
                             stop_words='english')
train_vector = vectorizer.fit_transform(train_X)
tfidf_df = pd.DataFrame(train_vector.toarray(), columns=vectorizer.get_feature_names())
tfidf_df

,accessory,acg,achille,acid,acler,aeyde,agence,agni,agolde,air,...,yolie,york,zadig,zebra,zermatt,zeynep,zimmermann,zip,zoom,étoile
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
test_vector = vectorizer.transform(X_test)

In [44]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

vector_model = LogisticRegression(C=12.0)
cols_target = ["cold weather","day to night","night out","vacation","weekend","work","workout"]
vector_result = pd.DataFrame(columns = cols_target, index = X_test.index)

for label in cols_target:
    print('Predicting {}'.format(label))
    train_y = y_train[label]
    vector_model.fit(train_vector, train_y)
    pred_train = vector_model.predict(train_vector)
    print('Training accuracy is {}'.format(accuracy_score(train_y, pred_train)))
    
    test_y_prob = vector_model.predict_proba(test_vector)[:,1]
    test_y = y_test[label]
    pred_test = vector_model.predict(test_vector)
    print('Testing accuracy is {}'.format(accuracy_score(test_y, pred_test)))
    vector_result[label] = test_y_prob
    filename = "{}".format(label)+"_vector_model.sav"
    pickle.dump(vector_model, open(filename, 'wb'))

Predicting cold weather
Training accuracy is 0.9790949986768986
Testing accuracy is 0.971604938271605
Predicting day to night
Training accuracy is 0.9010320190526594
Testing accuracy is 0.821604938271605
Predicting night out
Training accuracy is 0.9065890447208256
Testing accuracy is 0.8296296296296296
Predicting vacation
Training accuracy is 0.9367557554908706
Testing accuracy is 0.8814814814814815
Predicting weekend
Training accuracy is 0.9097644879597777
Testing accuracy is 0.8364197530864198
Predicting work
Training accuracy is 0.9203492987562847
Testing accuracy is 0.8401234567901235
Predicting workout
Training accuracy is 0.9936491135220958
Testing accuracy is 0.9839506172839506


In [45]:
vector_result

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e600eckpav431bzqjem5v4wy01e600eg7dzgakzjghgkrfvyb3,0.510627,0.956440,0.328873,0.001694,0.762661,0.902612,0.001063
01e1jm16ncm11qc1gsmkh3erch01e1jm16ns1wtkcph80zeswkx3,0.007869,0.530110,0.311398,0.072296,0.797711,0.094975,0.003338
01e2ky9h2267fxc9d382f077jd01e2ky9h4j0ffd2zcf826e8zm5,0.519297,0.937468,0.543377,0.003200,0.911415,0.302649,0.045014
01e4edbbqfaxnjdz93scvnnw8n01e4edbbqsx9zk71wd5vqe5yqe,0.001768,0.014783,0.003960,0.482873,0.881111,0.092727,0.001885
01e2m0qx4j85htkq2136q0tbvb01e2m0qx4xx4zpm06rj5888gzk,0.000805,0.775752,0.027485,0.122726,0.868422,0.030798,0.003711
...,...,...,...,...,...,...,...
01e2m4heymjm0f3ygprsxz8c0d01e2m4hez04zkv0h81824s23zt,0.000422,0.931602,0.070340,0.023082,0.450124,0.461679,0.000338
01e2kyfmf75neaxyhxc8vypad801e2kyfsrwnxya6y39c6d7gvk8,0.005459,0.933230,0.225142,0.028371,0.307340,0.682354,0.002641
01e4rts300d1qvsgtedt3kp7wz01e4rts309b93wvg0vj2krscwc,0.006875,0.802226,0.008146,0.017612,0.964517,0.019886,0.010089


_____

## 5. Combine embedding model and vector model together

In [46]:
# Get weighted-avg predicted probability from 2 models
combined_result = pd.DataFrame(index=vector_result.index,
                              columns=cols_target)
for label in cols_target:
    combined_result[label] = 0.4*embedding_result[label]+0.6*vector_result[label]

combined_result

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e600eckpav431bzqjem5v4wy01e600eg7dzgakzjghgkrfvyb3,0.315774,0.973677,0.197328,0.006572,0.857562,0.543146,0.000638
01e1jm16ncm11qc1gsmkh3erch01e1jm16ns1wtkcph80zeswkx3,0.008249,0.669247,0.324077,0.044058,0.710527,0.078049,0.002110
01e2ky9h2267fxc9d382f077jd01e2ky9h4j0ffd2zcf826e8zm5,0.694567,0.755411,0.329651,0.001945,0.693717,0.581372,0.027079
01e4edbbqfaxnjdz93scvnnw8n01e4edbbqsx9zk71wd5vqe5yqe,0.001772,0.009373,0.004315,0.583080,0.928050,0.055784,0.001131
01e2m0qx4j85htkq2136q0tbvb01e2m0qx4xx4zpm06rj5888gzk,0.000606,0.526744,0.019397,0.118779,0.793441,0.022946,0.242498
...,...,...,...,...,...,...,...
01e2m4heymjm0f3ygprsxz8c0d01e2m4hez04zkv0h81824s23zt,0.001056,0.958753,0.415305,0.024184,0.589748,0.380865,0.000291
01e2kyfmf75neaxyhxc8vypad801e2kyfsrwnxya6y39c6d7gvk8,0.035068,0.952776,0.170462,0.017081,0.197686,0.803680,0.001790
01e4rts300d1qvsgtedt3kp7wz01e4rts309b93wvg0vj2krscwc,0.004205,0.881332,0.005015,0.010609,0.963092,0.014392,0.006100


In [47]:
# Final Decision Threshold:
## If a product's probability of belonging to an occasion is >0.5, assign occasion value
## If none of occasion probability is >0.5 for a product, assign with occasion with highest probability

def decision(probs):
    if sum(probs>0.5)>0:
        probs[probs > 0.5] = 1
        probs[probs <= 0.5] = 0
    else:
        probs[probs == np.max(probs)] = 1
        probs[probs != np.max(probs)] = 0
    return probs

In [48]:
combined_result.apply(decision, axis=1)

,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,
01e600eckpav431bzqjem5v4wy01e600eg7dzgakzjghgkrfvyb3,0.0,1.0,0.0,0.0,1.0,1.0,0.0
01e1jm16ncm11qc1gsmkh3erch01e1jm16ns1wtkcph80zeswkx3,0.0,1.0,0.0,0.0,1.0,0.0,0.0
01e2ky9h2267fxc9d382f077jd01e2ky9h4j0ffd2zcf826e8zm5,1.0,1.0,0.0,0.0,1.0,1.0,0.0
01e4edbbqfaxnjdz93scvnnw8n01e4edbbqsx9zk71wd5vqe5yqe,0.0,0.0,0.0,1.0,1.0,0.0,0.0
01e2m0qx4j85htkq2136q0tbvb01e2m0qx4xx4zpm06rj5888gzk,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
01e2m4heymjm0f3ygprsxz8c0d01e2m4hez04zkv0h81824s23zt,0.0,1.0,0.0,0.0,1.0,0.0,0.0
01e2kyfmf75neaxyhxc8vypad801e2kyfsrwnxya6y39c6d7gvk8,0.0,1.0,0.0,0.0,0.0,1.0,0.0
01e4rts300d1qvsgtedt3kp7wz01e4rts309b93wvg0vj2krscwc,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [49]:
combined_result.columns = [str(col) + '_pred' for col in combined_result.columns]
combined_result = combined_result.astype(int)
combined_result

,cold weather_pred,day to night_pred,night out_pred,vacation_pred,weekend_pred,work_pred,workout_pred
id,,,,,,,
01e600eckpav431bzqjem5v4wy01e600eg7dzgakzjghgkrfvyb3,0,1,0,0,1,1,0
01e1jm16ncm11qc1gsmkh3erch01e1jm16ns1wtkcph80zeswkx3,0,1,0,0,1,0,0
01e2ky9h2267fxc9d382f077jd01e2ky9h4j0ffd2zcf826e8zm5,1,1,0,0,1,1,0
01e4edbbqfaxnjdz93scvnnw8n01e4edbbqsx9zk71wd5vqe5yqe,0,0,0,1,1,0,0
01e2m0qx4j85htkq2136q0tbvb01e2m0qx4xx4zpm06rj5888gzk,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...
01e2m4heymjm0f3ygprsxz8c0d01e2m4hez04zkv0h81824s23zt,0,1,0,0,1,0,0
01e2kyfmf75neaxyhxc8vypad801e2kyfsrwnxya6y39c6d7gvk8,0,1,0,0,0,1,0
01e4rts300d1qvsgtedt3kp7wz01e4rts309b93wvg0vj2krscwc,0,1,0,0,1,0,0


In [51]:
compare = pd.concat([combined_result, y_test], axis=1, sort=False)
compare

,cold weather_pred,day to night_pred,night out_pred,vacation_pred,weekend_pred,work_pred,workout_pred,cold weather,day to night,night out,vacation,weekend,work,workout
id,,,,,,,,,,,,,,
01e600eckpav431bzqjem5v4wy01e600eg7dzgakzjghgkrfvyb3,0,1,0,0,1,1,0,0,1,0,0,1,0,0
01e1jm16ncm11qc1gsmkh3erch01e1jm16ns1wtkcph80zeswkx3,0,1,0,0,1,0,0,0,1,0,0,1,0,0
01e2ky9h2267fxc9d382f077jd01e2ky9h4j0ffd2zcf826e8zm5,1,1,0,0,1,1,0,1,0,0,0,1,0,1
01e4edbbqfaxnjdz93scvnnw8n01e4edbbqsx9zk71wd5vqe5yqe,0,0,0,1,1,0,0,0,0,0,1,1,0,0
01e2m0qx4j85htkq2136q0tbvb01e2m0qx4xx4zpm06rj5888gzk,0,1,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01e2m4heymjm0f3ygprsxz8c0d01e2m4hez04zkv0h81824s23zt,0,1,0,0,1,0,0,0,1,0,0,0,1,0
01e2kyfmf75neaxyhxc8vypad801e2kyfsrwnxya6y39c6d7gvk8,0,1,0,0,0,1,0,0,1,0,0,0,1,0
01e4rts300d1qvsgtedt3kp7wz01e4rts309b93wvg0vj2krscwc,0,1,0,0,1,0,0,0,1,0,0,1,0,0


In [52]:
def accuracy(table):
    total_actual = len(compare)*7
    acc=0
    for index, row in table.iterrows():
        if row['cold weather_pred'] == row['cold weather']:
            acc+=1
        if row['day to night_pred'] == row['day to night']:
            acc+=1
        if row['night out_pred'] == row['night out']:
            acc+=1
        if row['vacation_pred'] == row['vacation']:
            acc+=1
        if row['weekend_pred'] == row['weekend']:
            acc+=1
        if row['work_pred'] == row['work']:
            acc+=1
        if row['workout_pred'] == row['workout']:
            acc+=1
    acc_rate = acc/total_actual
    return acc,acc_rate

In [53]:
accuracy(compare)

(10183, 0.8979717813051147)